In [30]:
import plotly.express as px
import pandas as pd
import numpy as np
from typing import Tuple
from sklearn import metrics
import os

In [31]:
#params:
path = '/home/mandy/project/facial_feature_impact_comparison/configs/ps_sr/results/'
result_path = '/home/mandy/project/facial_feature_impact_comparison/configs/ps_sr/results/all_aucs.csv'
num_ids = [2,5,10,50,100,200,500,960]
num_pics = [1,5,10,20,50,100,200,300]

In [32]:
# sorted(fc7_full.columns)
sorting = ['dom_01.jpg',
 'dom_02.jpg',
 'dom_03.jpg',
 'dom_04.jpg',
 'dom_05.jpg',
 'dom_06.jpg',
 'dom_07.jpg',
 'dom_09.jpg',
 'dom_10.jpg',
 'matt_01.jpg',
 'matt_02.jpg',
 'matt_03.jpg',
 'matt_04.jpg',
 'matt_05.jpg',
 'matt_06.jpg',
 'matt_07.jpg',
 'matt_08.jpg',
 'matt_09.jpg',
 'matt_10.jpg',
 'taz_01.jpg',
 'taz_02.jpg',
 'taz_03.jpg',
 'taz_04.jpg',
 'taz_05.jpg',
 'taz_06.jpg',
 'taz_07.jpg',
 'taz_08.jpg',
 'taz_09.jpg',
 'taz_10.jpg',
 'tess_01.jpg',
 'tess_02.jpg',
 'tess_03.jpg',
 'tess_04.jpg',
 'tess_05.jpg',
 'tess_06.jpg',
 'tess_07.jpg',
 'tess_08.jpg',
 'tess_09.jpg',
 'tess_10.jpg',
 'omer_02.jpg',
 'omer_03.jpg',
 'omer_04.jpg',
 'omer_05.jpg',
 'omer_06.jpg',
 'omer_07.jpg',
 'omer_08.jpg',
 'omer_09.jpg',
 'omer_10.jpg',
 'tal_01.jpg',
 'tal_02.jpg',
 'tal_03.jpg',
 'tal_04.jpg',
 'tal_05.jpg',
 'tal_06.jpg',
 'tal_07.jpg',
 'tal_08.jpg',
 'tal_09.jpg',
 'tal_10.jpg',
 'tamar_01.jpg',
 'tamar_02.jpg',
 'tamar_03.jpg',
 'tamar_04.jpg',
 'tamar_05.jpg',
 'tamar_06.jpg',
 'tamar_07.jpg',
 'tamar_08.jpg',
 'tamar_09.jpg',
 'tamar_10.jpg',
 'marina_01.jpg',
 'marina_02.jpg',
 'marina_03.jpg',
 'marina_04.jpg',
 'marina_05.jpg',
 'marina_06.jpg',
 'marina_07.jpg',
 'marina_08.jpg',
 'marina_09.jpg',
 'marina_10.jpg',
]


In [33]:

char_start_idx = []
pairs = [('dom', 'matt'), ('taz', 'tess'), ('omer', 'tal'), ('tamar', 'marina')]
for pair in pairs:
  for person in pair:
    i, _ = next((i, s) for (i, s) in enumerate(sorting) if s.startswith(person))
    char_start_idx.append(i)
print(char_start_idx)

[0, 9, 19, 29, 39, 48, 58, 68]


In [34]:
#auc defs
def calc_pair_auroc(data: pd.DataFrame, pair: Tuple[str,str]):
  pair_idx = data.columns.str.startswith(pair[0]) | data.columns.str.startswith(pair[1])
  
  pair_data = data.loc[pair_idx, pair_idx]
  pair_labels = true_df.loc[pair_idx, pair_idx]
  idx = np.tril_indices(pair_data.shape[0],-1)
  
  y_hat = 1 - pair_data.to_numpy()[idx]
  y = pair_labels.to_numpy()[idx]
  return metrics.roc_auc_score(y, y_hat)


In [35]:
#make col names:
files_name = []
for ids in num_ids:
    for image in num_pics:
        file_name = str(ids) +'_ids_' + str(ids) +'_'+ str(image) + '.csv'
        files_name.append(file_name)


In [36]:
#mean_AUC df:
col_names = ['ids_num', 'img_num', 'auc']
aucs_df = pd.DataFrame(0, index=np.arange(1), columns=files_name)


In [37]:
aucs_df

,2_ids_2_1.csv,2_ids_2_5.csv,2_ids_2_10.csv,2_ids_2_20.csv,2_ids_2_50.csv,2_ids_2_100.csv,2_ids_2_200.csv,2_ids_2_300.csv,5_ids_5_1.csv,5_ids_5_5.csv,...,500_ids_500_200.csv,500_ids_500_300.csv,960_ids_960_1.csv,960_ids_960_5.csv,960_ids_960_10.csv,960_ids_960_20.csv,960_ids_960_50.csv,960_ids_960_100.csv,960_ids_960_200.csv,960_ids_960_300.csv
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
aucs_df_final = pd.DataFrame(0, index=np.arange(64), columns=col_names)

In [39]:
#run on all models:
for ids in num_ids:
    for image in num_pics:
        file_name = str(ids) +'_ids_' + str(ids) +'_'+ str(image) + '.csv'
        full_path_to_file = os.path.join(path, file_name)
        print(file_name)
        #open csv:
        fc7_full = pd.read_csv(full_path_to_file)
        
    
        #add names column as index
        indices = fc7_full.columns.tolist()[1:]
        new_indices =[]
        for index in indices:
            index= index.replace('.jpg.1', '.jpg')
            new_indices.append(index)
        fc7_full.insert(0, 'Name', new_indices)
        fc7_full = fc7_full.drop(['Unnamed: 0'], axis = 1)
        # change Unnamed 0.1 to name 
        fc7_full.rename(columns = {'Unnamed: 0.1':'Name'}, inplace = True)

        #remove weird char from column name
        fc7_full.columns = fc7_full.columns.str.replace('.jpg.1', '.jpg')
        #rows:
        fc7_full = fc7_full.drop_duplicates(subset=['Name'])
        #remove duplicated columns:
        fc7_full = fc7_full.loc[:, ~fc7_full.columns.duplicated()]
        fc7_full.set_index('Name',inplace=True)
        index_list = list(fc7_full.index.values)
#         px.imshow(fc7_full, x=sorting, y=sorting).show()
        fc7_full.sort_index(axis=1, inplace=True)
        fc7_full.sort_index(axis=0, inplace=True)
        #true label mat
        true_df = fc7_full * 0
        for i in range(1,len(true_df.columns)+1):
            for z in range(1,len(true_df.index)+1):
                if (i//10) == (z//10):
#                     print(i,z)
                    true_df.iloc[i-1,z-1]=1
        
        #AUC
        aucs = {'pair': [], 'auroc': []}

        pair_country = ['UK','UK','IL','IL']

        # for pair, country in zip(pairs, pair_country):
        #   aucs['pair'].append(pair)
        #   aucs['country'].append(country)
        #   aucs['model'].append('CLIP')
        #   aucs['auroc'].append(calc_pair_auroc(clip_full, pair))

        for pair in (pairs):
          aucs['pair'].append(pair)
          aucs['auroc'].append(calc_pair_auroc(fc7_full, pair))


        # for pair, country in zip(pairs, pair_country):
        #   aucs['pair'].append(pair)
        #   aucs['country'].append(country)
        #   aucs['model'].append('FC7')
        #   aucs['auroc'].append(calc_pair_auroc(fc7_full, pair))

        aucs = pd.DataFrame(aucs)
#         print("aucs: ", aucs)
#         # aucs.to_csv('./drive/MyDrive/gylab/dp_sr/models_performance.csv')
#         # aucs = aucs.groupby(['country', 'model']).agg(
#         #     Mean_AUROC=('auroc', np.mean),
#         #     SD=('auroc', np.std)).reset_index()
        aucs = aucs.agg(Mean_AUROC=('auroc', np.mean),
            SD=('auroc', np.std)).reset_index()


        # print(aucs)
        # # fig = px.bar(pd.DataFrame(aucs), x='model', y='Mean_AUROC', color='country', barmode='group', text_auto=True, error_y='SD', template="plotly_white", width=700)
        # fig = px.bar(pd.DataFrame(aucs), x='model', y='Mean_AUROC', color='country', barmode='group', error_y='SD', template="plotly_white", width=700)
        # fig.show()
        aucs.set_index(['index'],inplace = True)
        aucs = aucs.T
        print(aucs[['Mean_AUROC']].to_string(index=False)) 
        aucs_df.at['0', file_name] = aucs['Mean_AUROC'].to_string(index= False)


        fig = px.bar(pd.DataFrame(aucs), x=aucs.index, y='Mean_AUROC', barmode='group', error_y='SD', template="plotly_white", width=700)
#         fig.show()
        print(aucs_df)




2_ids_2_1.csv
 Mean_AUROC
   0.519091
  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0            0.0             0.0             0.0   
0      0.519091            NaN             NaN             NaN   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0             0.0              0.0              0.0              0.0   
0             NaN              NaN              NaN              NaN   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0            0.0            0.0  ...                  0.0   
0            NaN            NaN  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 N

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000            0.0  ...                  0.0   
0       0.474661            NaN  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 NaN                 NaN   

   960_ids_

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000       0.000000  ...                  0.0   
0       0.474661       0.506061  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 NaN                 NaN   

   960_ids_

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000       0.000000  ...                  0.0   
0       0.474661       0.506061  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 NaN                 NaN   

   960_ids_

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000       0.000000  ...                  0.0   
0       0.474661       0.506061  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 NaN                 NaN   

   960_ids_

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000       0.000000  ...                  0.0   
0       0.474661       0.506061  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 NaN                 NaN   

   960_ids_

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000       0.000000  ...                  0.0   
0       0.474661       0.506061  ...                  NaN   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0                  0.0                0.0                0.0   
0                  NaN                NaN                NaN   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0                 0.0                 0.0                 0.0   
0                 NaN                 NaN                 NaN   

   960_ids_

  2_ids_2_1.csv  2_ids_2_5.csv  2_ids_2_10.csv  2_ids_2_20.csv  \
0             0       0.000000        0.000000        0.000000   
0      0.519091       0.508632        0.494559        0.477222   

   2_ids_2_50.csv  2_ids_2_100.csv  2_ids_2_200.csv  2_ids_2_300.csv  \
0        0.000000         0.000000         0.000000         0.000000   
0        0.476785         0.515356         0.523275         0.525822   

   5_ids_5_1.csv  5_ids_5_5.csv  ...  500_ids_500_200.csv  \
0       0.000000       0.000000  ...             0.000000   
0       0.474661       0.506061  ...             0.898368   

   500_ids_500_300.csv  960_ids_960_1.csv  960_ids_960_5.csv  \
0              0.00000           0.000000           0.000000   
0              0.89839           0.499439           0.585843   

   960_ids_960_10.csv  960_ids_960_20.csv  960_ids_960_50.csv  \
0            0.000000            0.000000            0.000000   
0            0.680217            0.815344            0.902996   

   960_ids_

In [40]:
type(aucs_df.iloc[[1]].T.squeeze())

pandas.core.series.Series

In [41]:
aucs_df_final['auc'] = aucs_df.iloc[[1]].T.squeeze().to_list()
aucs_df_final['ids_num']=[2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,10,10,10,10,10,10,10,10,50,50,50,50,50,50,50,50,100,100,100,100,100,100,100,100,200,200,200,200,200,200,200,200,500,500,500,500,500,500,500,500,960,960,960,960,960,960,960,960]
aucs_df_final['img_num']=[1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300,1,5,10,20,50,100,200,300]

In [42]:
aucs_df_final

,ids_num,img_num,auc
0,2,1,0.519091
1,2,5,0.508632
2,2,10,0.494559
3,2,20,0.477222
4,2,50,0.476785
...,...,...,...
59,960,20,0.815344
60,960,50,0.902996
61,960,100,0.903109
62,960,200,0.904595


In [43]:
#save csv
aucs_df_final.to_csv(result_path)
